In [2]:
#GPU count and name
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-aa33b99a-b8bd-60b9-de0e-4d72e0b2d721)


In [3]:
!pip uninstall Keras tensorflow -y
!pip install Keras==2.4 tensorflow==2.3

Uninstalling Keras-2.4.3:
  Successfully uninstalled Keras-2.4.3
Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 174kB 8.1MB/s 
     |████████████████████████████████| 320.4MB 54kB/s 
     |████████████████████████████████| 460kB 45.3MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0


In [ ]:
!python -c 'import keras; print(keras.__version__)'

2021-03-02 12:40:45.944342: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.
2.4.0


## Download and extract dataset

In [5]:
import os

# replace with your own Kaggle username and key

os.environ['KAGGLE_USERNAME'] = "alfieroddan" 
os.environ['KAGGLE_KEY'] = "3d51d62e7a492d1a66279a64fbba29b2" 

!kaggle datasets download kbhartiya83/swimming-pool-and-car-detection -p /content/data --unzip 

 94% 104M/111M [00:00<00:00, 92.8MB/s] 
100% 111M/111M [00:01<00:00, 82.4MB/s]


## Clone Retinanet Repo

In [6]:
import os

repo_url = 'https://github.com/alfieroddan/keras-retinanet.git'
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url))) 

!git clone {repo_url} # clone repo

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6211 (delta 1), reused 5 (delta 1), pack-reused 6205
Receiving objects: 100% (6211/6211), 13.47 MiB | 35.74 MiB/s, done.
Resolving deltas: 100% (4202/4202), done.


In [7]:
# from /content/keras-retinanet
%cd /content/keras-retinanet 
!pip install .
!python setup.py build_ext --inplace

%cd ..

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp37-cp37m-linux_x86_64.whl size=168009 sha256=d43b45f4a966b09c0a1a25de90c8c7d989acf31286c2a5d8cc8322203f42888b
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=96eb7f221c30d221585f1142c96d4fd71673e3461337497a69c9191c2e53c095
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-retinanet keras-resnet
running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/k

## Prepare dataset for training

In [8]:
import os
base_dir = os.getcwd()
base_dir

'/content'

In [9]:
# select subsample of N for initial training

from shutil import copyfile
import random

# determine number for subsample
NUM = 2000

# create dir for subsample
sub_dir = base_dir + '/images_subset/'
!mkdir {sub_dir}

image_dir = base_dir + '/data/training_data/training_data/images/'
image_paths = os.listdir(image_dir)
random_NUM = random.sample(image_paths, NUM) # randomly select subsample 

# copy subsample into subsample dir
for i in random_NUM:
    copyfile(image_dir + i, sub_dir + i )

In [10]:
%cd /content/images_subset/
!ls -F |grep -v / | wc -l

/content/images_subset
2000


In [11]:
!python /content/keras-retinanet/build_dataset.py \
-l /content/data/training_data/training_data/labels/ \
-i /content/images_subset/ \
-r /content/images_subset/train.csv \
-e /content/images_subset/test.csv \
-c /content/images_subset/classes.csv 

[INFO] creating 'train' set ...
[INFO] 1600 total images in 'train'
[INFO] writing train annotations ...
[INFO] total 7068 annotations
[INFO] train.csv completed
[INFO] creating 'test' set ...
[INFO] 400 total images in 'test'
[INFO] writing test annotations ...
[INFO] total 1714 annotations
[INFO] test.csv completed
[INFO] writing classes ...
[INFO] classes.csv completed
[FINAL] Task completed!


## Download Pretrained Model

In [15]:
import urllib.request

PRETRAINED_MODEL = '/content/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'

urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

('/content/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5',
 <http.client.HTTPMessage at 0x7f50a116a210>)

## Train CNN

In [12]:
!wc -l /content/images_subset/train.csv

7068 /content/images_subset/train.csv


In [13]:
# compute number of annotations to compute number of steps per epoch

import pandas as pd
import math

df = pd.read_csv('/content/images_subset/train.csv', header=None)
count = len(df)
no_steps = math.floor(count/32)

print("Count of annotations: {}".format(count))
print("Number of steps per epoch: {}".format(no_steps))

Count of annotations: 7068
Number of steps per epoch: 220


In [16]:
!mkdir /content/run_1/
%cd /content/run_1/
#If _TensorLike not right change _Tensorlike to Tensor
!python /content/keras-retinanet/keras_retinanet/bin/train.py \
--freeze-backbone \
--random-transform \
--weights {PRETRAINED_MODEL} \
--weighted-average \
--batch-size 32 \ #No Images/batch size (?)
--steps 50 \
--epochs 30 \
csv '/content/images_subset/train.csv' '/content/images_subset/classes.csv'

mkdir: cannot create directory ‘/content/run_1/’: File exists
/content/run_1
2021-03-03 11:08:14.432270: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.
Creating model, this may take a second...
2021-03-03 11:08:15.768475: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-03-03 11:08:15.797129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-03 11:08:15.797701: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-03 11:08:15.797739: I tensorflow/stream_executor/platform/default/

/content/run_1/snapshots/resnet50_csv_30.h5


## Evaluate model

In [1]:
# path to trained weights
import os
model_1_path = os.path.join('/content/run_1/snapshots/', sorted(os.listdir('/content/run_1/snapshots/'), reverse=True)[0])
print(model_1_path)

# evaluate model on test.csv
!python /content/keras-retinanet/keras_retinanet/bin/evaluate.py \
csv '/content/images_subset/test.csv' \
'/content/images_subset/classes.csv' \
{model_1_path} --convert-model

%cd ..

FileNotFoundError: ignored

## Generate detections on test images - ANNOTATION FILES

In [ ]:
# create directory with selected test images 

!mkdir /content/data/test_data_images/test_data_images/test
!mv -t /content/data/test_data_images/test_data_images/test \
/content/data/test_data_images/test_data_images/images/000000091.jpg \

mkdir: cannot create directory ‘/content/data/test_data_images/test_data_images/test’: File exists
mv: cannot stat '/content/data/test_data_images/test_data_images/images/000000091.jpg': No such file or directory


## Convert training model to inference model

In [ ]:
# print trained model path
model_1_path

'/content/run_1/snapshots/resnet50_csv_30.h5'

In [ ]:
os.getcwd()

'/content'

In [ ]:
# convert trained model to inference model to be used for generating predictions
!python /content/keras-retinanet/keras_retinanet/bin/convert_model.py '/content/run_1/snapshots/resnet50_csv_30.h5' '/content/run_1/snapshots/resnet50_csv_30_inference.h5'
inference_model = '/content/run_1/snapshots/resnet50_csv_30_inference.h5'

2021-03-02 13:55:23.818676: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-03-02 13:55:25.015665: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-03-02 13:55:25.044255: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-02 13:55:25.044854: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-02 13:55:25.044897: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-03-02 13:55:25.047891: I tensorflow/stream_executor/platform/default

In [ ]:
# generate prediction files
!python /content/keras-retinanet/image_inference_write.py \
-i '/content/data/test_data_images/test_data_images/test/' \
-t 0.6 \
-m {inference_model} \
-o  /content/data/ \
-l /content/images_subset/classes.csv

2021-03-02 14:11:40.823973: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.
2021-03-02 14:11:42.555455: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-03-02 14:11:42.586873: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-02 14:11:42.587452: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-02 14:11:42.587498: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-03-02 14:11:42.589223: I tensorflow/stream

In [ ]:
# combine all csv files into one
import glob
fout=open("/content/data/output/out.csv","a")
fout.write("path,confidence,ymin,xmin,ymax,xmax,class\n") # create header
for fi in glob.glob("/content/data/*.csv"):
  f = open(fi)
  for line in f:
    fout.write(line)
fout.close()

## Generate detections on test images - IMAGES

In [ ]:
os.getcwd()

'/content'

In [ ]:
# create output directory where you want to save images with bounding boxes
!mkdir /content/data/output

# generate detections on images
!python /content/keras-retinanet/image_inference_print.py \
-i /content/data/test_data_images/test_data_images/test/ \
-t 0.2 \
-m  {inference_model} \
-o /content/data/output

mkdir: cannot create directory ‘/content/data/output’: File exists
2021-03-02 14:13:13.190940: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.
2021-03-02 14:13:15.015972: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-03-02 14:13:15.047938: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-02 14:13:15.048540: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-02 14:13:15.048590: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic librar

In [ ]:
from google.colab import files

# download output images
for p in glob.glob('/content/data/output/*.jpg'):
  files.download(p)
  
# download raw images
for p in glob.glob('/content/data/test_data_images/test_data_images/test/*.jpg'):
  files.download(p)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# create a dataframe for analysis purposes

import pandas as pd
df = pd.read_csv("/content/data/output/out.csv")
df.head()

,path,score,box_1,box_0,box_3,box_2,label
0,/content/data/test_data_images/test_data_image...,0.857935,145.444930,210.636300,156.79843,222.177800,1
1,/content/data/test_data_images/test_data_image...,0.784386,50.129322,35.037167,96.69036,79.764015,2
2,/content/data/test_data_images/test_data_image...,0.771489,132.898540,208.894960,144.47559,220.212630,1
3,/content/data/test_data_images/test_data_image...,0.685553,171.916490,210.330720,183.58807,221.704570,1
4,/content/data/test_data_images/test_data_image...,0.658636,171.934220,145.900200,183.75516,157.409680,1


In [ ]:
df["label"].value_counts()

1    20
2     3
Name: label, dtype: int64